In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

MODEL_NAME = "gpt-4o-mini"

In [17]:
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [18]:
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [19]:
def ask_question(question, role, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "system", "content": role}, {"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [ ]:
def handle_tool_call(response, role, question):
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [21]:
def process_response(question, role, tools):
    response = ask_question(question, role, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        final_response = handle_tool_call(response, role, question)
        return final_response.choices[0].message.content.strip()
    else:
        return response.choices[0].message.content.strip()

In [22]:
tools = define_tools()

role = input("キャラクターを設定:")

messages=[]

while(True):
    question = input("メッセージを入力:")
    if question.strip()=="":
        break
    display(f"質問:{question}")

    messages.append({"role": "user", "content": question.strip()})
    if len(messages) > 8:
        del_message = messages.pop(0)

    response_message = process_response(question, role, tools)

    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")


---ご利用ありがとうございました！---
